In [1]:
# warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from crewai import Agent, Task, Crew, Process

### Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import DuckDuckGoSearchResults

In [7]:
# initialize tools
scrape_tool = ScrapeWebsiteTool()
tavily_search_tool = TavilySearchResults(max_results=5)
duck_search_tool = DuckDuckGoSearchResults(source="news", num_results=5)

### Agents

In [9]:
# Agent 1: Industry Analyst
industry_analyst = Agent(
    role="Senior Industry Analyst",
    goal="Search {expected_num} global companies in {industry} industry "
         "that have business presence in {region} with operations "
         "involving any of the following: [production, manufacturing, supply chain], "
         "whose products and service offerings using {material} as major raw material. ",
    tools=[tavily_search_tool, duck_search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
        "With seasoned knowledge and extrodinary understanding "
        "of {material} usage across industries, "
        "you excel at finding companies in {industry}"
        "that perfectly fit the criteria of usage of {material} "
        "for their products and service offerings."
    ),
    allow_delegation=False,
)

In [10]:
# Agent 2: Product and Operations Specialist
product_specialist = Agent(
    role='Product Specialist',
    goal=(
        "Verify for each of the company, that their products and service offering " 
        "that INDEED heavily use the material: {material}, "
        "based on the preliminary list from the Industry Analyst agent."
    ),
    tools=[tavily_search_tool, duck_search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
        "This agent is fact-driven and detail-oriented, highly organized, "
        "it ensures that for each company, their products and service offerings "
        "are actually heavily using the {material} "
        "via flawless examination."
    ),
    allow_delegation=False,
    max_iter=15,
)

In [11]:
# Agent 3: Financial Analyst
financial_analyst = Agent(
    role='Senior Financial Analst',
    goal=(
        "Monitor and analyze {material} market data in real-time "
        "at global and {region} level, "
        "to identify trends and predict market movements."
    ),
    tools=[scrape_tool, tavily_search_tool, duck_search_tool, ],
    verbose=True,
    backstory=("You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
              "Specializing in financial markets analytics, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "this Agent is the cornerstone for "
              "informing trading insights."
    ),
    allow_delegation=False,
    max_iter=15,
)

In [12]:
# Agent 4: FX options sales leader
FX_CIO = Agent(
    role="Chief FX Options Investment Officer",
    goal="Effectively market the FX Options offering and "
         "communicate with the potential clients, received from Operations Specialist agent",

    verbose=True,
    backstory=("You work for HSBC MSS division that provides versatile financial offerings to mid-large clients."
              "Equipped with a deep understanding of financial "
              "markets and unparallel FX investment experience, this agent "
              "devises and refines trading strategies that are tailored for each potential clients." 
              "It evaluates the performance of different approaches to determine "
              "the most profitable and risk-averse options.\n"
              "Creative and communicative, "
              "you craft compelling and engaging messages for the potential clients. "
              "Your role is pivotal in converting interest "
              "into action, guiding leads through the journey "
              "from curiosity to commitment."
    ),
    allow_delegation=False,
    max_iter=15,
)

### Create Venue Pydanic Object

In [13]:
from pydantic import BaseModel, Field, InstanceOf
from typing import Optional, List

class LeadsDetails(BaseModel):
    company: str
    product: list[str] = []
    material: str
    region: str = ""
    operations: list[str] = []
    region_status: bool = False

class LeadsAll(BaseModel):
    companies: list[LeadsDetails]

In [14]:
LeadsDetails.model_json_schema()

{'properties': {'company': {'title': 'Company', 'type': 'string'},
  'product': {'default': [],
   'items': {'type': 'string'},
   'title': 'Product',
   'type': 'array'},
  'material': {'title': 'Material', 'type': 'string'},
  'region': {'default': '', 'title': 'Region', 'type': 'string'},
  'operations': {'default': [],
   'items': {'type': 'string'},
   'title': 'Operations',
   'type': 'array'}},
 'required': ['company', 'material'],
 'title': 'LeadsDetails',
 'type': 'object'}

### Tasks

In [15]:
industrial_task = Task(
    tools=[tavily_search_tool, duck_search_tool, ],
    description="Search {expected_num} companies in {industry} industry "
                "that they have operations in {region} "
                "involving any of the following: [production, manufacturing, supply chain], "
                "which heavily use physical {material} as raw material "
                "to produce products or service offerings. ",
    expected_output="List of {expected_num} companies, "
                    "with short descriptions of each company's products or service offerings "
                    "made with {material} as raw material "
                    "and their operations in {region}",
      
    agent=industry_analyst,
)

In [16]:
product_task = Task(
    tools=[scrape_tool, tavily_search_tool, duck_search_tool,],
    
    description="Generate potential client list "
                "based on the initial industry list, "
                "verify each company that their products and service offerings " 
                "are INDEED heavily using {material}. "    ,
    expected_output="List ONLY the companies that are verified, "
                    "with short descriptions of each company's products or service offerings "
                    "made with {material} as raw material",

    context=[industrial_task],
    agent=product_specialist,
)

In [17]:
analytic_task = Task(
    tools=[scrape_tool, tavily_search_tool, duck_search_tool,],
    
    description="Continuously monitor and analyze market data for "
                "the underlying asset ({material}). "
                "Use statistical modeling and machine learning to "
                "identify trends and predict both global and {region} market movements."
    ,
    expected_output="Insights extracted from most recent significant market "
                    "opportunities or risks for {material} price movement.",

    context=[industrial_task],
    agent=financial_analyst,
)

In [18]:
advisory_task = Task(
    description="Based on each potential client list, "
                "and financial insights from the {material} market data analysis, "
                "Promote HSBC MSS {offering} "
                "for the underlying asset {material} "
                "aiming to engage the potential clients."
                "If no potential client has operations in {region}, terminate the task. "
                "Else"
                "For every single potential client, do the following: "
                    "Using the knowledge gathered from "
                    "the potential client's information of their operations in {region} "
                    "and their products and services offerings, "
                    "alongside with the {material} market analytics insights, "
                    "craft a tailored outreach campaign "
                    "aimed at key decision makers. "
                    "The campaign should address their products and operations in {region}, "
                    "financial insights from most recent market opportunities or risks from {material} price movement, "
                    "and how HSBC MSS {offering} can be their best of interest. "
                "Your communication must resonate "
                "with company values and their downstream customers, "
                "demonstrating a deep understanding of "
                "their business and needs.\n"
                "Don't make assumptions and only "
                "use information you absolutely sure about.",
    
    expected_output="A series of professional email drafts in markdown format, "
                    "tailored to every optential client. "
                    "Each draft should include "
                    "a compelling narrative that connects our {offering} "
                    "with their productions and operations. "
                    "Ensure the tone is engaging, professional, "
                    "and aligned with each client's corporate identity.",

    context=[product_task, analytic_task],
    output_file="_04.sales_offerings_3.md",  # Outputs the report as a text file
    agent=FX_CIO,
)

### Crew

In [19]:
# Define the crew with agents and tasks
financial_leads_crew = Crew(
    agents=[industry_analyst,
           product_specialist,
           financial_analyst,
           FX_CIO],
    
    tasks=[industrial_task, 
           product_task, 
           analytic_task,
           advisory_task],

    process=Process.sequential,
    
    verbose=2,

    max_rpm=2,
)

In [20]:
leads_details = {
    'expected_num': 3,
    'industry': "Electronics",
    'material': "silver",
    'region': "China",
    "offering": "FX options",  # NDF : FX Forward
}

In [21]:
import time

class RateLimitError(Exception):
    pass
    
def call_model(leads_details):
    try:
        response = financial_leads_crew.kickoff(inputs=leads_details)
        
        # Check if the response indicates a rate limit error and raise RateLimitError if so
        if 'error' in response and response['error']['code'] == 'rate_limit_exceeded':
            raise RateLimitError(response)
        
        # Process the response if no rate limit error
        print("Model call successful\n")
        return response
    
    except RateLimitError as e:
        # Re-raise the rate limit error to be handled by the retry function
        raise e
    except Exception as e:
        # Handle other potential exceptions from the model call
        print(f"An unexpected error occurred: {e}")
        raise e

def call_crew_with_retry(leads_details):
    try:
        return call_model(leads_details)
    except RateLimitError as e:
        # Extract the wait time from the error message
        wait_time = float(e.args[0]['error']['message'].split('try again in ')[1].split('s.')[0])
        print(f"Rate limit exceeded. Retrying in {wait_time} seconds.")
        time.sleep(wait_time)
        return call_crew_with_retry(leads_details)

In [22]:
%%time
result = call_crew_with_retry(leads_details)

 [DEBUG]: == Working Agent: Senior Industry Analyst
 [INFO]: == Starting Task: Search 3 companies in Electronics industry that they have operations in China involving any of the following: [production, manufacturing, supply chain], which heavily use physical silver as raw material to produce products or service offerings. 


> Entering new CrewAgentExecutor chain...
I need to find 3 global companies in the Electronics industry that have business presence in China, with operations involving production, manufacturing, or supply chain, and heavily use physical silver as a raw material.

Action: tavily_search_results_json
Action Input: {"query": "Electronics companies in China using silver as raw material"} 

[{'url': 'https://www.yicaiglobal.com/news/20230608-10-chinas-fusion-new-material-soars-on-plan-to-produce-silver-powder-for-solar-cells', 'content': "Fusion New Material's share price [SHA:688503] was trading up 5.4 percent at CNY74.28 (USD10.40) as of 1.30 p.m. China time today. Ear

In [24]:
from IPython.display import Markdown
Markdown("_04.sales_offerings_3.md")

Here are the three email drafts tailored to each potential client:

**Email Draft for Fusion New Material**

**Subject: Mitigate Silver Price Risks with HSBC MSS FX Options**

Dear [Key Decision Maker's Name],

I hope this email finds you well. As a leading producer of electronic-grade silver powder, Fusion New Material is no stranger to the volatility of the silver market. Our analysis suggests that the current silver price of $31.71 per ounce is subject to fluctuations, with a predicted range of $30 to $35 per ounce in the short term.

At HSBC MSS, we understand that managing silver price risks is crucial to your business operations and profitability. Our FX options can help you hedge against potential losses and lock in profit margins. With our expertise, you can focus on what matters most - producing high-quality silver powder for the solar panel industry.

Our FX options can be tailored to your specific needs, providing a hedge against silver price fluctuations. This can help you:

* Stabilize your production costs
* Protect your profit margins
* Focus on growth and innovation

We would be delighted to discuss how HSBC MSS FX options can support your business goals. Please let us know if you're interested in exploring this opportunity further.

Best regards,

[Your Name]

**Email Draft for JA Solar Holdings Co., Ltd.**

**Subject: Protect Your Profit Margins with HSBC MSS FX Options**

Dear [Key Decision Maker's Name],

As a leading manufacturer of photovoltaic (PV) panels, JA Solar Holdings Co., Ltd. is well aware of the importance of managing silver price risks. Our analysis suggests that the silver market will continue to be volatile in the short term, with prices potentially ranging between $30 and $35 per ounce.

At HSBC MSS, we believe that our FX options can help you mitigate the impact of silver price fluctuations on your business. By hedging against potential losses, you can protect your profit margins and focus on delivering high-quality PV panels to your customers.

Our FX options can be tailored to your specific needs, providing a hedge against silver price fluctuations. This can help you:

* Stabilize your production costs
* Protect your profit margins
* Focus on growth and innovation

We would be delighted to discuss how HSBC MSS FX options can support your business goals. Please let us know if you're interested in exploring this opportunity further.

Best regards,

[Your Name]

**Email Draft for Trina Solar Co., Ltd.**

**Subject: Manage Silver Price Risks with HSBC MSS FX Options**

Dear [Key Decision Maker's Name],

As a leading manufacturer of solar panels, Trina Solar Co., Ltd. is no stranger to the importance of managing silver price risks. Our analysis suggests that the silver market will continue to be volatile in the short term, with prices potentially ranging between $30 and $35 per ounce.

At HSBC MSS, we believe that our FX options can help you mitigate the impact of silver price fluctuations on your business. By hedging against potential losses, you can protect your profit margins and focus on delivering high-quality solar panels to your customers.

Our FX options can be tailored to your specific needs, providing a hedge against silver price fluctuations. This can help you:

* Stabilize your production costs
* Protect your profit margins
* Focus on growth and innovation

We would be delighted to discuss how HSBC MSS FX options can support your business goals. Please let us know if you're interested in exploring this opportunity further.

Best regards,

[Your Name]